### Quantizing an LLM

- Any LLM > Quantize > GGUF format
- use on your own CPU
- LLM had billions of parameters, so we need adoption, not everyone had GPU.
- GGUF is a format that allows to run LLM on CPU/GPU and more (compression teqniques i.e. Quantization)

In [ ]:
# we will use llama.cpp
!!git clone https://github.com/ggerganov/llama.cpp

["Cloning into 'llama.cpp'...",
 'remote: Enumerating objects: 26304, done.\x1b[K',
 'remote: Counting objects:   0% (1/6272)\x1b[K',
 'remote: Counting objects:   1% (63/6272)\x1b[K',
 'remote: Counting objects:   2% (126/6272)\x1b[K',
 'remote: Counting objects:   3% (189/6272)\x1b[K',
 'remote: Counting objects:   4% (251/6272)\x1b[K',
 'remote: Counting objects:   5% (314/6272)\x1b[K',
 'remote: Counting objects:   6% (377/6272)\x1b[K',
 'remote: Counting objects:   7% (440/6272)\x1b[K',
 'remote: Counting objects:   8% (502/6272)\x1b[K',
 'remote: Counting objects:   9% (565/6272)\x1b[K',
 'remote: Counting objects:  10% (628/6272)\x1b[K',
 'remote: Counting objects:  11% (690/6272)\x1b[K',
 'remote: Counting objects:  12% (753/6272)\x1b[K',
 'remote: Counting objects:  13% (816/6272)\x1b[K',
 'remote: Counting objects:  14% (879/6272)\x1b[K',
 'remote: Counting objects:  15% (941/6272)\x1b[K',
 'remote: Counting objects:  16% (1004/6272)\x1b[K',
 'remote: Counting objects:  17% (

In [ ]:
# Use CUBLAS - is a software that help you to decide what machines are using
# I am having a GPU
!cd llama.cpp && LLAMA_CUBLAS=1 make && pip install -r requirements.txt

I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE -DGGML_USE_CUDA -I/usr/local/cuda/include -I/usr/local/cuda/targets/x86_64-linux/include -DGGML_CUDA_USE_GRAPHS  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -fopenmp -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread -fopenmp  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE -DGGML_USE_CUDA -I/usr/local

In [ ]:
# huggingface snapshot download
# download, and convert into 4-bit quantized model
# compressed teqniques - 1-bit for inference LLM (reduced computation power)

from huggingface_hub import snapshot_download

In [ ]:
model_name = "Qwen/Qwen1.5-1.8B"

In [ ]:
# define methods i.e. list
# you can pass 4-bits and others, which will create diffrent files

methods = ["q4_k_m"]

In [ ]:
base_model = "./original_model/"
quantized_path = "./quantized_model/"

In [ ]:
# downlaod the base model
snapshot_download(
    repo_id = model_name,
    local_dir = base_model,
    local_dir_use_symlinks = False
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1194: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/7.28k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

'/content/original_model'

In [ ]:
original_model = quantized_path+'/FP16.gguf'

In [ ]:
!mkdir ./quantized_model/

In [ ]:
# convert-hf-to-gguf.py
# block-wise quantization
!python llama.cpp/convert-hf-to-gguf.py ./original_model/ --outtype f16 --outfile ./quantized_model/FP16.gguf


INFO:hf-to-gguf:Loading model: original_model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Set model parameters
INFO:hf-to-gguf:gguf: context length = 32768
INFO:hf-to-gguf:gguf: embedding length = 2048
INFO:hf-to-gguf:gguf: feed forward length = 5504
INFO:hf-to-gguf:gguf: head count = 16
INFO:hf-to-gguf:gguf: key-value head count = 16
INFO:hf-to-gguf:gguf: rope theta = 1000000.0
INFO:hf-to-gguf:gguf: rms norm epsilon = 1e-06
INFO:hf-to-gguf:gguf: file type = 1
INFO:hf-to-gguf:Set model tokenizer
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
INFO:gguf.vocab:Adding 151387 merge(s).
INFO:gguf.vocab:Setting special token type eos to

### Type 0 Quant:

w = d*q

- d: block scale

### Type 1 Quant:

w = d*q+m

- m: block minimum

In [ ]:
import os

In [ ]:
# applying methods
for m in methods:
  qtype = f"{quantized_path}/{m.upper()}.gguf"
  os.system("./llama.cpp/quantize "+quantized_path+"/FP16.gguf "+qtype+" "+m)



In [ ]:
# take the new quantized model, rag with chat-with-bob.txt
! ./llama.cpp/main -m ./quantized_model/Q4_K_M.gguf -n 90 --repeat_penalty 1.0 --color -i -r "User:" -f llama.cpp/prompts/chat-with-bob.txt

Log start
main: build = 3089 (c90dbe02)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: seed  = 1717560785
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from ./quantized_model/Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.name str              = original_model
llama_model_loader: - kv   2:                          qwen2.block_count u32              = 24
llama_model_loader: - kv   3:                       qwen2.context_length u32              = 32768
llama_model_loader: - kv   4:                     qwen2.embedding_length u32              = 2048
llama_model_loader: - kv   5:                  qwen2.feed_forward_length u32              = 5504
llama_model_loader: - kv   

In [ ]:
# Huggingface token

from huggingface_hub import notebook_login
notebook_login()

### Push quantized model to huggingface

In [ ]:
from huggingface_hub import HfApi, HfFolder, create_repo, upload_file

In [ ]:
# push model to huggingface, make it public access

model_path = "./quantized_model/Q4_K_M.gguf"
repo_name = "quantized1-qwen1.5-1.88B-GGUF"
repo_url = create_repo(repo_name, private=False)

In [ ]:
# push our quantized model files to huggingface

api = HfApi()

In [ ]:
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="Q4_K_M.gguf",
    repo_id = "amyang176/quantized1-qwen1.5-1.88B-GGUF",
    repo_type="model"
)

Q4_K_M.gguf:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/amyang176/quantized1-qwen1.5-1.88B-GGUF/commit/7959803f49e93a41e477ef9e6a7211246f107ed2', commit_message='Upload Q4_K_M.gguf with huggingface_hub', commit_description='', oid='7959803f49e93a41e477ef9e6a7211246f107ed2', pr_url=None, pr_revision=None, pr_num=None)